In [1]:
import pandas as pd
import numpy as np
import glob
import pathlib
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

Read the list of tickers from [https://www.sec.gov/file/company-tickers](https://www.sec.gov/file/company-tickers).

The current SEC guidelines limit users to a total of no more than 10 requests per second, regardless of the number of machines used to submit requests.

A user-agent must be declared in the request header in the following format:

```
Sample Company Name AdminContact@<sample company domain>.com
```

In [2]:
import urllib.request, json
import requests

sec_request_headers = {
    'User-Agent': 'University of Illinois ypark32@illinois.edu',
}

In [3]:
tickers_url = 'https://www.sec.gov/files/company_tickers.json'

response = requests.get(tickers_url, headers=sec_request_headers)
cik_dict = response.json()

Print the first five key-value pairs.

In [4]:
{k: cik_dict[k] for k in list(cik_dict)[:5]}

{'0': {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'},
 '1': {'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'},
 '2': {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'},
 '3': {'cik_str': 1652044, 'ticker': 'GOOGL', 'title': 'Alphabet Inc.'},
 '4': {'cik_str': 1018724, 'ticker': 'AMZN', 'title': 'AMAZON COM INC'}}

In [5]:
df = pd.DataFrame(
    data=cik_dict.values()
)

In [6]:
df

,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,1045810,NVDA,NVIDIA CORP
2,789019,MSFT,MICROSOFT CORP
3,1652044,GOOGL,Alphabet Inc.
4,1018724,AMZN,AMAZON COM INC
...,...,...,...
10190,1496254,LTAFX,Alternative Strategies Income Fund
10191,1496254,LTCFX,Alternative Strategies Income Fund
10192,1995306,SPHIF,SUPER HI INTERNATIONAL HOLDING LTD.
10193,1472033,JTGLF,"Citibank,N.A./ADR"


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10195 entries, 0 to 10194
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cik_str  10195 non-null  int64 
 1   ticker   10195 non-null  object
 2   title    10195 non-null  object
dtypes: int64(1), object(2)
memory usage: 239.1+ KB


The `"cik_str"` is an entity's Central Index Key (CIK). To use CIK values inside a query, the key must be zero-padded to be 10-digits long.

Pad the values in the `"cik_str"` column.

In [8]:
df['cik_str'] = df['cik_str'].astype(str).str.pad(width=10, fillchar='0')
df.head(10)

,cik_str,ticker,title
0,0000320193,AAPL,Apple Inc.
1,0001045810,NVDA,NVIDIA CORP
2,0000789019,MSFT,MICROSOFT CORP
3,0001652044,GOOGL,Alphabet Inc.
4,0001018724,AMZN,AMAZON COM INC
5,0001326801,META,"Meta Platforms, Inc."
6,0001067983,BRK-B,BERKSHIRE HATHAWAY INC
7,0001046179,TSM,TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD
8,0001730168,AVGO,Broadcom Inc.
9,0000059478,LLY,ELI LILLY & Co


In [9]:
AAPL_cik = df.query("ticker == 'AAPL'").iloc[0]['cik_str']

AAPL_cik

'0000320193'

### Company Facts

In [13]:
company_facts_url = f'https://data.sec.gov/api/xbrl/companyfacts/CIK{AAPL_cik}.json'

print('company_facts_url')
print(company_facts_url)

response = requests.get(company_facts_url, headers=sec_request_headers)
my_response = response.json()

# print(my_response)

type(my_response)

company_facts_url
https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json


dict